In [1]:
pip install requests beautifulsoup4 lxml

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\hp\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime,date
import pandas as pd


START_DATE = date(2022, 5, 15)    # our start fortnight
END_DATE   = date(2025, 8, 15)    # our end fortnight

BASE_URL = ("https://www.fpi.nsdl.co.in/web/StaticReports/"
        "Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_")

In [3]:

def request_html(url = "https://www.fpi.nsdl.co.in/web/StaticReports/Fortnightly_Sector_wise_FII_Investment_Data/FIIInvestSector_June152022.html"):
    session = requests.Session()
    session.headers.update({
        "User-Agent": "Mozilla/5.0 (compatible; LearningScraper/1.0)"
    })
    resp = session.get(url, timeout=8)

    return [resp.status_code,resp.text]

In [4]:
def header_row_extractor_from_html(html_doc):

    soup = BeautifulSoup(html_doc,'lxml')
    table = soup.find("table")
    rows = table.find_all("tr")
    i = 2
    j=32
    data_per_html_page = [[],[],[]]
    row_counter = 0
    data_per_html_page[0].append("Date")
    for tr in rows:
        tds = tr.find_all("td")
        if row_counter == 0:
            
            data_per_html_page[1].append(str_to_date(tds[2].get_text(strip=True)[10:]))
            data_per_html_page[2].append(str_to_date(tds[5].get_text(strip=True)[10:]))
            
        if row_counter>2 and len(tds)>j:
            data_per_html_page[0].append(tds[1].get_text(strip=True))            
            data_per_html_page[1].append(tds[i].get_text(strip=True))
            data_per_html_page[2].append(tds[j].get_text(strip=True))
        row_counter += 1
            
    return data_per_html_page

In [5]:

def data_extractor_from_html(html_doc):

    soup = BeautifulSoup(html_doc,'lxml')
    table = soup.find("table")
    rows = table.find_all("tr")
    i = 2
    j=32
    data_per_html_page = [[],[]]
    row_counter = 0
    for tr in rows:
        tds = tr.find_all("td")
        if row_counter == 0:
            
            data_per_html_page[0].append(str_to_date(tds[2].get_text(strip=True)[10:]))
            data_per_html_page[1].append(str_to_date(tds[5].get_text(strip=True)[10:]))
            
        if row_counter>2 and len(tds)>j:
            data_per_html_page[0].append(tds[i].get_text(strip=True))
            data_per_html_page[1].append(tds[j].get_text(strip=True))
        row_counter += 1
            
    return data_per_html_page



In [6]:
def data_extractor_from_html_sept_15_onwards(html_doc):
    soup = BeautifulSoup(html_doc,'lxml')
    table = soup.find("table")
    rows = table.find_all("tr")
    i = 2
    j=74
    data_per_html_page = [[],[]]
    row_counter = 0
    for tr in rows:
        tds = tr.find_all("td")
        if row_counter == 0:
            
            data_per_html_page[0].append(str_to_date(tds[2].get_text(strip=True)[10:]))
            data_per_html_page[1].append(str_to_date(tds[5].get_text(strip=True)[10:]))
            
        if row_counter>3 and len(tds)>j:
            data_per_html_page[0].append(tds[i].get_text(strip=True))
            data_per_html_page[1].append(tds[j].get_text(strip=True))
        row_counter += 1
            
    return data_per_html_page

In [7]:
def str_to_date(date_str):
    dt = datetime.strptime(date_str,"%B %d, %Y")
    return dt.date()


In [8]:
def adding_to_complete_data(complete_data,data_per_html_page):
    for row in data_per_html_page: 
        complete_data.append(row)

In [9]:
def url_variants(d,base_url):
    """Return possible URL formats for a given date based on month length rules."""
    month_name = d.strftime("%B")   # Full month name (e.g., June, September)
    dd   = str(d.day)               # day without leading zero
    yyyy = d.strftime("%Y")

    # Rule: if month name ≤ 4 letters, use full name; else use first 3 letters
    if int(yyyy) != 2023 and len(month_name) <= 4:
        mon_part = month_name
    else:
        mon_part = month_name[:3]   # e.g., Aug, Sep, Oct, Nov, Dec

    return [
        f"{base_url}{mon_part}{dd}{yyyy}.html"
    ]

In [10]:
def mid_month_dates(start, end):
    """Yield only the 15th of each month between start..end."""
    y, m = start.year, start.month
    while True:
        fifteenth = date(y, m, 15)
        if start <= fifteenth <= end:
            yield fifteenth
        if y > end.year or (y == end.year and m == end.month):
            break
        m += 1
        if m == 13:
            m, y = 1, y + 1

In [11]:
def dated_urls_for_range(start,end,base_url):
    seen = set()  # avoid duplicate if variants collide
    for d in mid_month_dates(start, end):
        for u in url_variants(d, base_url):
            if u not in seen:
                seen.add(u)
                yield d, u

In [12]:
def iterations(start,end):
    complete_data=[]
    for d, url in dated_urls_for_range(start, end, BASE_URL):
        html_doc = request_html(url)[1]
        if d == start:
            adding_to_complete_data(complete_data,header_row_extractor_from_html(html_doc))
        elif d < date(2024, 9, 15):
            adding_to_complete_data(complete_data,data_extractor_from_html(html_doc))            
        else:
            adding_to_complete_data(complete_data,data_extractor_from_html_sept_15_onwards(html_doc))
    return complete_data

In [13]:
def to_csv(complete_data, path="nsdl_sectorwise_auc.csv"):
    # split header + rows
    cols = complete_data[0]
    rows = complete_data[1:]

    df = pd.DataFrame(rows, columns=cols)

    # Date column
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

    # numeric columns: remove commas/whitespace, blank/dash to NA, then to Int64
    num_cols = [c for c in df.columns if c != "Date"]
    for c in num_cols:
        s = df[c].astype(str) \
                 .str.replace(",", "", regex=False) \
                 .str.strip() \
                 .replace({"": None, "None": None, "-": None, "—": None, "–": None})
        df[c] = pd.to_numeric(s, errors="coerce").astype("Int64")

    df.to_csv(path, index=False, date_format="%Y-%m-%d")
    return df


In [14]:
complete_data = iterations(START_DATE, END_DATE)
to_csv(complete_data)

,Date,Automobile and Auto Components,Capital Goods,Chemicals,Construction,Construction Materials,Consumer Durables,Consumer Services,Diversified,Fast Moving Consumer Goods,...,"Oil, Gas & Consumable Fuels",Power,Realty,Services,Telecommunication,Textiles,Utilities,Sovereign,Others,Grand Total
0,2022-04-30,221576,97186,102913,57832,79797,182883,110503,1608,277854,...,588240,237667,57853,74645,115643,21635,413,0,11227,4666454
1,2022-05-15,206363,87430,92336,51600,71862,163339,90377,1399,267224,...,522715,204696,49873,64993,105874,19201,346,0,10152,4231472
2,2022-05-31,228813,94510,97732,54644,70495,160593,102462,1469,278116,...,555923,199855,52991,72431,108740,20479,372,0,10425,4411991
3,2022-06-15,220158,92798,91631,51817,63271,147510,94901,1481,258961,...,545683,187532,50316,69091,103657,18647,333,0,15736,4150609
4,2022-06-30,229758,92721,87412,50669,63013,143724,90969,1546,265865,...,538924,195851,49682,65848,102684,17992,336,0,9470,4132029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2025-06-15,490153,379132,131258,124748,114767,186335,289708,4652,377744,...,515705,243253,156167,161120,333911,24798,2582,0,179804,7151924
76,2025-06-30,510983,394124,137226,127820,122722,194034,313844,4531,381081,...,542676,247251,154561,172192,367820,26087,2480,0,55438,7418858
77,2025-07-15,509003,392113,135592,122434,125173,193422,307682,4433,392609,...,540234,249685,154130,176066,355140,26297,2456,0,52444,7364330
78,2025-07-31,501618,379369,137733,122926,124187,189879,322786,4504,398986,...,505294,240892,138735,171375,349013,25645,2493,0,50970,7196613
